## 1. Import Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pyrsm as rsm

sns.set_style("darkgrid")

# increase plot resolution
mpl.rcParams["figure.dpi"] = 150

## 2. Import DataFrames

In [2]:
# import SBA combined data
df = pd.read_csv('combined.csv')
# remove the format column
df.drop(columns='Unnamed: 0', inplace=True)
# fill null value with 'N' in NonProfit column
df['NonProfit'] = df['NonProfit'].fillna('N')
#df.dtypes

/Users/robin/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (17,51) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# import NAICS code reference
naics = pd.read_csv('6-digit_2017_Codes.csv')
# remove the format column
naics.drop(columns='Unnamed: 2', inplace=True)
# rename columns
naics.rename(columns={"2017 NAICS Code": "NAICSCode", '2017 NAICS Title':'Industry'}, inplace=True)
# change Code into Integer
naics.loc[:,'NAICSCode'] = naics.loc[:,'NAICSCode'].astype('int')
# change code into String
naics.loc[:,'NAICSCode'] = naics.loc[:,'NAICSCode'].astype('str')

In [4]:
# fill null value with 0 in NAICSCode column
df['NAICSCode'] = df['NAICSCode'].fillna(0)
# change NAICS Code into Integer
df.loc[:,'NAICSCode'] = df.loc[:,'NAICSCode'].astype('int')
# change variables into categorical
df.loc[:,['SBAOfficeCode', 'ServicingLenderLocationID', 'NAICSCode', 'OriginatingLenderLocationID']]\
    = df.loc[:,['SBAOfficeCode', 'ServicingLenderLocationID', 'NAICSCode', 'OriginatingLenderLocationID']].astype('str')

In [5]:
# merge SBA and NAICS reference
df = df.merge(naics, on='NAICSCode', how='left')

## 3. Summarize Data

In [6]:
# define categorical and numerical variables lists
cat = []
num = []
# loop in all columns in the DataFrame
for col in df.columns:
    # set condition that data type equals object
    if df[col].dtype == object:
        # append to categorical variable list if True
        cat.append(col)
    else:
        # append to numerical variable list if False
        num.append(col)

In [7]:
# build numerical data info DataFrame
num_df = pd.DataFrame({'Column Name':num,
                       '# of Records':np.NaN,
                       '% populated':np.NaN,
                       'Unique Values':np.NaN,
                       'Mean':np.NaN,
                       'Standard Deviation':np.NaN,
                       'Minimum Value':np.NaN,
                       'Maximum Value':np.NaN,
                       '# of Zeros':np.NaN})
# calculate summary info
for i in range(len(num)):
    num_df.loc[i,'# of Records'] = df[num[i]].count()
    num_df.loc[i,'% populated'] = num_df['# of Records'][i]/len(df)*100
    num_df.loc[i,'Unique Values'] = len(df[num[i]].dropna().unique())
    num_df.loc[i,'Mean'] = df[num[i]].mean()
    num_df.loc[i,'Standard Deviation'] = df[num[i]].std()
    num_df.loc[i,'Minimum Value'] = df[num[i]].min()
    num_df.loc[i,'Maximum Value'] = df[num[i]].max()
    num_df.loc[i,'# of Zeros'] = (df[num[i]] == 0).sum(axis=0)
# show summary info
num_df
# save summary info into csv
num_df.to_csv('num_df.csv')

In [8]:
# build categorical data info DataFrame
cat_df = pd.DataFrame({'Column Name':cat,
                       '# of Records':np.NaN,
                       '% populated':np.NaN,
                       'Unique Values':np.NaN,
                       'Most Common':np.NaN})
# calculate summary info
for i in range(len(cat)):
    cat_df.loc[i,'# of Records'] = df[cat[i]].count()
    cat_df.loc[i,'% populated'] = cat_df['# of Records'][i]/len(df)*100
    cat_df.loc[i,'Unique Values'] = len(df[cat[i]].dropna().unique())
    cat_df.loc[i,'Most Common'] = df[cat[i]].mode()[0]
# For unique value, don't show mode value
for i in range(len(cat_df)):
    if cat_df.loc[i,'# of Records'] == cat_df['Unique Values'][i]:
        cat_df.loc[i,'Most Common'] = '-'
# show summary info_df
cat_df
# save summary info into csv
cat_df.to_csv('cat_df.csv')

## 4. Visulizing Data

In [ ]:
# boxplot for each categorical variable
for i in cat:
    sns.catplot(x=i, kind="count", palette="crest", data=df, order=df[i].value_counts().iloc[:8].index)\
       .set(xlabel=i, ylabel='Count')\
       .set_xticklabels(rotation=90)
    plt.savefig("df/"+i+".png", bbox_inches = "tight")

In [ ]:
# histplot for each numerical variable
for i, col in enumerate(num):
    plt.figure(i)
    sns.histplot(data=df, x=col, color="darkseagreen")\
       .set(xlabel=col, ylabel='Count')
    plt.savefig("df/num/"+col+".png", bbox_inches = "tight")

In [ ]:
# example for boxplot single categorical variable
sns.catplot(x="ProcessingMethod", kind="count", data=df, order=df['ProcessingMethod'].value_counts().index)\
   .set(xlabel='Processing Method', ylabel='Count')#\
   #.set_xticklabels(rotation=90)
#plt.savefig("figure.png", bbox_inches = "tight")

In [ ]:
# calculate included data in histplot in percentage
len(ecbg[ecbg.Shape__Len <= 2000000])/len(ecbg)

## 5. Aggregate Data

In [9]:
# make a list of variables that need to be groupby
groupby_list = ['Term', 
                'SBAOfficeCode', 
                'ProcessingMethod', 
                'BorrowerCity', 
                'BorrowerState', 
                'LoanStatus', 
                'FranchiseName', 
                'ServicingLenderCity', 
                'ServicingLenderState', 
                'RuralUrbanIndicator', 
                'HubzoneIndicator', 
                'LMIIndicator', 
                'BusinessAgeDescription', 
                'ProjectCity', 
                'ProjectCountyName', 
                'ProjectState', 
                'CD', 
                'Industry', 
                'Race', 
                'Ethnicity', 
                'BusinessType', 
                'OriginatingLender', 
                'OriginatingLenderCity', 
                'OriginatingLenderState', 
                'Gender', 
                'Veteran', 
                'NonProfit']
# make a list of variables that need to be calculated after groupby
num_list = ['InitialApprovalAmount',
            'CurrentApprovalAmount',
            'UndisbursedAmount', 
            'UTILITIES_PROCEED', 
            'PAYROLL_PROCEED', 
            'MORTGAGE_INTEREST_PROCEED',
            'RENT_PROCEED', 
            'REFINANCE_EIDL_PROCEED', 
            'HEALTH_CARE_PROCEED', 
            'DEBT_INTEREST_PROCEED', 
            'ForgivenessAmount']

In [10]:
# aggregate dataframe
for i in groupby_list:
    agg_df = df.groupby([i])[num_list].sum().reset_index()
    agg_df['count'] = agg_df[i].map(df[i].value_counts())
    agg_df.to_csv('agg_df/'+i+'.csv')